# Import libraries <a id = "0"></a> 

In [14]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import seaborn as sns

from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")


In [15]:
df = pd.read_csv('./amazon_reviews.csv')


In [16]:
# Loại bỏ các cột không cần thiết
df = df.drop(columns=['verified', 'image', 'reviewTime', 'summary', 'reviewText', 'vote'])
# Loại bỏ các dòng có giá trị NaN trong cột 'price'
df= df.dropna(subset=['price'])
df.head() # Hiển thị 5 dòng đầu tiên của DataFrame

,userName,itemName,description,brand,feature,category,price,rating
0,Amazon Customer,"Toblerone Swiss Milk Chocolate Bar, Crunchy Sa...","[""Made from deliciously decadent ingredients, ...",Toblerone,[],Prime_Pantry,$1.63,5.0
1,Carla,"Prince of Peace Organic Tea, Oolong, 100 Tea Bags","['Prince of Peace Enterprises, Inc., founded i...",Prince Of Peace,[],Grocery_and_Gourmet_Food,$6.40,5.0
2,Jaclyn,"Pet Champion Adjustable No-Pull Harness, Colla...","[""The Pet Champion Large/ Extra Large 22-36 in...",Pet Champion,"['Features Bright Pink Lemonade pattern', 'Dua...",Pet_Supplies,$7.99,5.0
3,Christinne M,Koh-I-Noor Progresso Woodless Colored 24-Penci...,['Koh-I-Noor Progresso Woodless Colored 24-Pen...,KOH-I-NOOR,"['Rich, pigmented, solid color encased in lacq...",Arts_Crafts_and_Sewing,$14.18,5.0
6,patrick,Blue Buffalo Life Protection Formula Natural P...,['Because puppyhood is such an important stage...,Blue Buffalo,['HIGH QUALITY PUPPY FOOD: Blue Buffalo always...,Pet_Supplies,$28.99,5.0


In [17]:
# Đếm số lần xuất hiện của mỗi userName và lọc
user_counts = df['userName'].value_counts()
valid_usernames = user_counts[user_counts >= 30].index

# Đếm số lần xuất hiện của mỗi itemName và lọc
item_counts = df['itemName'].value_counts()
valid_items = item_counts[item_counts >= 5].index

# Lọc dữ liệu dựa trên userName và itemName
df_filtered = df[~df['userName'].isin(['Amazon Customer']) & 
                 df['userName'].isin(valid_usernames) &
                 df['itemName'].isin(valid_items)]

In [18]:
#shape of the dataframe

df_filtered.shape

(49389, 8)

In [19]:
df = df_filtered.copy()

In [20]:
# Lấy danh sách duy nhất các userName
users = df[['userName']].drop_duplicates().reset_index(drop=True)
# Tạo userId tự tạo
users['userId'] = users.index + 1  # Bắt đầu từ 1

# Chọn cột cần thiết
users = users[['userId', 'userName']]

# Lưu vào tệp CSV
users.to_csv('users.csv', index=False)


In [21]:
# Lấy danh sách duy nhất các itemName và feature
items = df[['itemName']].drop_duplicates().reset_index(drop=True)

# Tạo itemId tự tạo
items['itemId'] = items.index + 1  # Bắt đầu từ 1

# Chọn cột cần thiết
items = items[['itemId', 'itemName']]

# Lưu vào tệp CSV
items.to_csv('items.csv', index=False)


In [22]:

# Đọc lại tệp users và items để đảm bảo
users = pd.read_csv('users.csv')
items = pd.read_csv('items.csv')

# Merge để lấy userId và itemId
ratings = df.merge(users, on='userName', how='left')
ratings = ratings.merge(items, on='itemName', how='left')

# Chọn các cột cần thiết
ratings = ratings[['userId', 'itemId', 'rating']]

# Bước 1: Sắp xếp dữ liệu theo cột 'userId' tăng dần
ratings_sorted = ratings.sort_values(by='userId', ascending=True)

# Kiểm tra dữ liệu sau khi sắp xếp
print("\nDữ liệu sau khi sắp xếp:")
print(ratings_sorted.head())

# Bước 2: Lưu dữ liệu đã sắp xếp vào tệp 'ratings.csv'
ratings_sorted.to_csv('ratings.csv', index=False)



Dữ liệu sau khi sắp xếp:
       userId  itemId  rating
0           1       1     5.0
42685       1    8428     5.0
42826       1     910     5.0
29490       1   11762     5.0
18162       1    8604     5.0


In [23]:
items_df= pd.read_csv('./items.csv')
items_df.head()


,itemId,itemName
0,1,"Prince of Peace Organic Tea, Oolong, 100 Tea Bags"
1,2,Cimo Slim Flexible TPU Cimo Shock Absorbing Ca...
2,3,PetSafe Easy Walk Harness
3,4,Kaytee Crittertrail Lazy Lookout Accessory Kit
4,5,Ethical Pet Wide Durable Heavy Gauge Plastic C...


In [24]:
ratings_df= pd.read_csv('./ratings.csv')
ratings_df.head()


,userId,itemId,rating
0,1,1,5.0
1,1,8428,5.0
2,1,910,5.0
3,1,11762,5.0
4,1,8604,5.0


In [25]:
users_df = pd.read_csv('./users.csv')
users_df.head()
n_users = users.shape[0]


In [26]:
#Example of Stemming
from nltk.stem import PorterStemmer
porter = PorterStemmer()
porter.stem('dancing')

import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [27]:
import pandas as pd
import re
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
# Chuyển đổi cột itemName thành chữ thường
items_df["itemName"] = items_df["itemName"].str.lower()

# Hàm loại bỏ thẻ (tag)
def remove_tags(text):
    pattern = re.compile("<.*?>")
    return pattern.sub(r'', text)

# Xóa thẻ
items_df['itemName'] = items_df['itemName'].apply(remove_tags)

# Xóa dấu câu
punctuation = string.punctuation

def remove_punc(text):
    return text.translate(str.maketrans('', '', punctuation))

items_df['itemName'] = items_df['itemName'].apply(remove_punc)

# Sử dụng CountVectorizer và tính cosine similarity
cv = CountVectorizer(max_features=5000, stop_words="english")
vectors = cv.fit_transform(items_df["itemName"]).toarray()
similarity = cosine_similarity(vectors)

In [29]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [30]:
print("TF-IDF của item đầu tiên:", vectors[0])

TF-IDF của item đầu tiên: [0 0 0 ... 0 0 0]


In [31]:
# Lấy tên các đặc trưng
feature_names = cv.get_feature_names_out()
print("Feature names:", feature_names)


Feature names: ['000' '003' '03' ... 'zukes' 'zupreem' 'zymox']


In [32]:
# Tạo DataFrame từ tên các đặc trưng
features_df = pd.DataFrame(feature_names, columns=['Feature Names'])

# Lưu DataFrame thành file CSV
features_df.to_csv('feature_names.csv', index=False)

print("Feature names have been saved to 'feature_names.csv'")

Feature names have been saved to 'feature_names.csv'


In [33]:
# Kiểm tra gợi ý

# Hàm gợi ý (recommend)
def recommend(tag):
    item_index = items_df[items_df["itemName"] == tag].index[0]
    distances = similarity[item_index]
    item_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    for i in item_list:
        print(items_df.iloc[i[0]].itemName)


recommend("petstages tower of tracks cat toy")

afc petmate 26317 cat crazies cat toy
moody pet flingamastring cat toy
neko flies kragonfly interchangeable cat toy attachment only
petstages cat scratcher cat hammock cat scratching post
cat dancer  cat charmer wand teaser toy


In [34]:
# Chuyển đổi DataFrame thành Numpy array
ratings_matrix = ratings.to_numpy()


In [35]:
def get_items_rated_by_user(rate_matrix, user_id):
    """
    Trả về (item_ids, scores)
    """
    user_row_indices = np.where(rate_matrix[:, 0] == user_id + 1)[0]
    if len(user_row_indices) == 0:
        return np.array([]), np.array([])  # Nếu không có đánh giá, trả về mảng rỗng

    item_ids = rate_matrix[user_row_indices, 1] - 1  # Chuyển đổi thành chỉ số bắt đầu từ 0
    scores = rate_matrix[user_row_indices, 2]
    return item_ids.astype(int), scores  # Chuyển đổi thành kiểu số nguyên


In [36]:
n_users = users.shape[0]
print("Number of users:", n_users)
# users.head()  # Uncomment để xem một vài ví dụ

Number of users: 908


In [37]:
# Khởi tạo danh sách để lưu trữ mô hình KNN cho mỗi người dùng
knn_models = [None] * n_users

In [38]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsRegressor
from math import sqrt

In [39]:
n_items = items_df.shape[0]

In [41]:
import numpy as np
from math import sqrt

# Giả sử bạn đã định nghĩa `ratings_df` và chuyển đổi nó thành mảng NumPy
ratings_matrix = ratings_df.to_numpy()

# Hàm lấy các item đã được người dùng đánh giá
def get_items_rated_by_user(rate_matrix, user_id):
    """
    Trả về (item_ids, scores)
    """
    user_row_indices = np.where(rate_matrix[:, 0] == user_id + 1)[0]
    if len(user_row_indices) == 0:
        return np.array([]), np.array([])  # Nếu không có đánh giá, trả về mảng rỗng

    item_ids = rate_matrix[user_row_indices, 1] - 1  # Chuyển đổi thành chỉ số bắt đầu từ 0
    scores = rate_matrix[user_row_indices, 2]
    return item_ids.astype(int), scores  # Chuyển đổi thành kiểu số nguyên

# Khởi tạo danh sách để lưu trữ mô hình KNN cho mỗi người dùng
knn_models = [None] * n_users

# Huấn luyện KNN Regressor cho mỗi người dùng
for n in range(n_users):
    ids, scores = get_items_rated_by_user(ratings_matrix, n)

    if len(ids) == 0:
        knn_models[n] = None  # Không có đánh giá
        continue

    knn = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='cosine')

    # Đảm bảo rằng ids là kiểu số nguyên
    Xhat = vectors[ids.astype(int), :]  # Đặc trưng của các item đã được đánh giá
    y = scores  # Điểm đánh giá thực tế

    # Huấn luyện mô hình
    knn.fit(Xhat, y)

    # Lưu mô hình
    knn_models[n] = knn

n_items = items_df.shape[0]

# Dự đoán điểm đánh giá cho tất cả người dùng và item
Yhat = np.zeros((n_items, n_users))

# Hàm đánh giá MAE
def evaluate(Yhat, rates):
    absolute_errors = 0  # Tổng lỗi tuyệt đối
    count = 0  # Số lượng đánh giá

    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        if len(ids) == 0:
            continue  # Nếu người dùng chưa đánh giá item nào, bỏ qua

        scores_pred = Yhat[ids, n]
        errors = np.abs(scores_truth - scores_pred)  # Lỗi tuyệt đối
        absolute_errors += errors.sum()  # Cộng dồn lỗi tuyệt đối
        count += errors.size  # Cộng dồn số lượng đánh giá

    return absolute_errors / count if count != 0 else float('nan')  # Tính MAE

# In kết quả MAE
print("MAE for training:", evaluate(Yhat, ratings_matrix))


MAE for training: 4.340521168681286
